In [0]:
dbutils.widgets.dropdown("environment", "prd", ["prd"], "1-Environment")
env = dbutils.widgets.get("environment")

# events

In [0]:
query = f"""
create or replace table {env}_silver.event.events
using delta
as
select  
    id, 
    name,
    locale,
    url, 
    pleaseNote as please_note,
    from_utc_timestamp(dates.start.dateTime, 'UTC') as date_start_utc,
    from_utc_timestamp(dates.end.dateTime, 'UTC') as date_end_utc,
    dates.timezone as date_timezone,
    dates.status.code as date_status,
    dates.spanMultipleDays as date_span_multiple_days,
    ticketing.safeTix.enabled as ticketing_safe_tix_enabled,
    ticketing.allInclusivePricing.enabled as ticketing_all_inclusive,
    seatmap.staticUrl as seatmap,
    accessibility.ticketLimit as ticket_limit,
    ageRestrictions.legalAgeEnforced as legal_age_enforced,
    priceRanges[0].currency as price_currency,
    priceRanges[0].min as price_min,
    priceRanges[0].max as price_max,
    sales.public.startDateTime as sales_start,
    sales.public.endDateTime as sales_end,
    sales.public.startTBD as sales_tbd,
    sales.public.startTBA as sales_tba,
    sales.presales[0].startDateTime as presales_start,
    sales.presales[0].endDateTime as presales_end,
    sales.presales[0].name as presales_name
from {env}_bronze.api_ticketmaster.events 
where type = "event"
  and test is false
  and id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.events alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.events add constraint pk_event_id primary key (id)"
spark.sql(query)

DataFrame[]

# images

In [0]:
query = f"""
create or replace table {env}_silver.event.image as 
with events_cte as (
  select 
    id, 
    images
  from {env}_bronze.api_ticketmaster.events
  where 
    type = "event"
    and test is false
    and id is not null
), exploded_cte as (
  select 
    id as id_event, 
    explode(images) as image
  from events_cte
), distinct_cte as (
  select distinct
    id_event,
    image.ratio as ratio,
    image.url as url,
    image.width as width,
    image.height as height,
    image.fallback as fallback
  from exploded_cte
)
select 
  sha2(
    concat_ws(
      '||',
      coalesce(id_event, ''),
      coalesce(ratio, ''),
      coalesce(url, ''),
      coalesce(cast(width as string), ''),
      coalesce(cast(height as string), ''),
      coalesce(cast(fallback as string), '')
    ), 
    256
  ) as id,
  id_event,
  ratio, 
  url, 
  width, 
  height, 
  fallback
from distinct_cte
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.image alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.image add constraint pk_image_id primary key (id)"
spark.sql(query)

DataFrame[]

In [0]:
query = f"""
alter table {env}_silver.event.image 
add constraint fk_image_id_event 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)

DataFrame[]

# classifications

In [0]:
query = f"""
create or replace temp view temp_classification as
with events_cte as (
  select 
    id, 
    classifications 
  from {env}_bronze.api_ticketmaster.events
  where 
    type = 'event'
    and test is false
    and id is not null
), exploded_cte as (
  select 
    id as id_event, 
    explode(classifications) as classification
  from events_cte
)
select 
  id_event, 
  classification 
from exploded_cte
"""

spark.sql(query)

DataFrame[]

## segment

In [0]:
query = f"""
create or replace table {env}_silver.event.segment as
select distinct
  classification.segment.id, 
  classification.segment.name
from temp_classification
where classification.segment.id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.segment alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.segment add constraint pk_segment_id primary key (id)"
spark.sql(query)

DataFrame[]

## genre

In [0]:
query = f"""
create or replace table {env}_silver.event.genre as
select distinct
  classification.genre.id, 
  classification.genre.name
from temp_classification
where classification.genre.id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.genre alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.genre add constraint pk_genre_id primary key (id)"
spark.sql(query)

DataFrame[]

## type

In [0]:
query = f"""
create or replace table {env}_silver.event.type as
select distinct
  classification.type.id, 
  classification.type.name
from temp_classification
where classification.type.id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.type alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.type add constraint pk_type_id primary key (id)"
spark.sql(query)

DataFrame[]

## sub genre

In [0]:
query = f"""
create or replace table {env}_silver.event.sub_genre as
select distinct
  classification.subGenre.id, 
  classification.subGenre.name, 
  classification.genre.id as id_genre
from temp_classification
where classification.subGenre.id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.sub_genre alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.sub_genre add constraint pk_sub_genre_id primary key (id)"
spark.sql(query)

DataFrame[]

In [0]:
query = f"alter table {env}_silver.event.sub_genre alter column id set not null"
spark.sql(query)

try:
    query = f"""
    alter table {env}_silver.event.sub_genre 
    drop constraint fk_sub_genre_genre_id
    """
    spark.sql(query)
except:
  pass


query = f"""
alter table {env}_silver.event.sub_genre 
add constraint fk_sub_genre_genre_id 
foreign key (id_genre) 
references {env}_silver.event.genre(id)
"""
spark.sql(query)

DataFrame[]

## sub type

In [0]:
query = f"""
create or replace table {env}_silver.event.sub_type as
select distinct
  classification.subType.id, 
  classification.subType.name, 
  classification.type.id as id_type
from temp_classification
where classification.subType.id is not null
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
query = f"alter table {env}_silver.event.sub_type alter column id set not null"
spark.sql(query)

query = f"alter table {env}_silver.event.sub_type add constraint pk_sub_type_id primary key (id)"
spark.sql(query)

DataFrame[]

In [0]:
query = f"alter table {env}_silver.event.sub_type alter column id set not null"
spark.sql(query)

try:
  query = f"""
  alter table {env}_silver.event.sub_type 
  drop constraint fk_sub_type_type_id
  """
  spark.sql(query)
except:
  pass

query = f"""
alter table {env}_silver.event.sub_type 
add constraint fk_sub_type_type_id 
foreign key (id_type) 
references {env}_silver.event.type(id)
"""
spark.sql(query)

DataFrame[]

# event_segment

In [0]:
query = f"""
create or replace table {env}_silver.event.event_segment as
select distinct
  id_event,
  classification.segment.id as id_segment,
  classification.subGenre.id as id_sub_genre,
  classification.subType.id as id_sub_type
from temp_classification
"""
spark.sql(query)

try:
    query = f"alter table {env}_silver.event.event_segment drop constraint fk_event_segment_event_id"
    spark.sql(query)
except:
    pass
try:
    query = f"alter table {env}_silver.event.event_segment drop constraint fk_event_segment_segment_id"
    spark.sql(query)
except:
    pass
try:
    query = f"alter table {env}_silver.event.event_segment drop constraint fk_event_segment_sub_genre_id"
    spark.sql(query)
except:
    pass
try:
    query = f"alter table {env}_silver.event.event_segment drop constraint fk_event_segment_sub_type_id"
    spark.sql(query)
except:
    pass

query = f"""
alter table {env}_silver.event.event_segment 
add constraint fk_event_segment_event_id 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_segment 
add constraint fk_event_segment_segment_id 
foreign key (id_segment) 
references {env}_silver.event.segment(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_segment 
add constraint fk_event_segment_sub_genre_id 
foreign key (id_sub_genre) 
references {env}_silver.event.sub_genre(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_segment 
add constraint fk_event_segment_sub_type_id 
foreign key (id_sub_type) 
references {env}_silver.event.sub_type(id)
"""
spark.sql(query)

DataFrame[]

# promoters

In [0]:
query = f"""
create or replace temp view temp_promoters as 
  select id as id_event, explode(promoters) as promoter
  from {env}_bronze.api_ticketmaster.events 
  where promoters is not null
"""
spark.sql(query)

DataFrame[]

In [0]:
query = f"""
create or replace table {env}_silver.event.promoter as
select  promoter.id as id, 
        first(promoter.name) as name, 
        first(promoter.description) as description 
from temp_promoters
group by promoter.id
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.promoter 
alter column id set not null
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.promoter 
add constraint pk_promoter_id primary key (id)
"""
spark.sql(query)

DataFrame[]

In [0]:
query = f"""
create or replace table {env}_silver.event.event_promoter as
select  distinct
        id_event,
        promoter.id as id_promoter
from temp_promoters
"""
spark.sql(query)

try:
    query = f"alter table {env}_silver.event.event_promoter drop constraint fk_event_promoter_event_id"
    spark.sql(query)
except:
    pass
try:
    query = f"alter table {env}_silver.event.event_promoter drop constraint fk_event_promoter_promoter_id"
    spark.sql(query)
except:
    pass

query = f"""
alter table {env}_silver.event.event_promoter 
add constraint fk_event_promoter_event_id 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_promoter 
add constraint fk_event_promoter_promoter_id 
foreign key (id_promoter) 
references {env}_silver.event.promoter(id)
"""
spark.sql(query)

DataFrame[]

# venues

In [0]:
query = f"""
create or replace temp view temp_venues as
with event_venues as (
    select  distinct
            id as id_event,
            _embedded.venues as vanues
    from 
      prd_bronze.api_ticketmaster.events
)
select id_event, explode(vanues) as vanue
from event_venues
"""
spark.sql(query)

DataFrame[]

## venues

In [0]:
query = f"""
create or replace table {env}_silver.event.venues as
select
  vanue.id as id,
  vanue.name as name,
  vanue.locale as locale,
  vanue.postalCode as postal_code,
  vanue.timezone as timezone,
  vanue.city.name as city,
  vanue.state.name as state,
  vanue.state.stateCode as state_code,
  vanue.country.name as country,
  vanue.country.countryCode as country_code,
  vanue.address.line1 as address,
  vanue.location.longitude as longitude,
  vanue.location.latitude as latitude,
  vanue.upcomingEvents.archtics as upcoming_events_archtics,
  vanue.upcomingEvents.ticketmaster as upcoming_events_ticketmaster,
  vanue.upcomingEvents._total as upcoming_events_total,
  vanue.upcomingEvents._filtered as upcoming_events_filtered,
  vanue.url as url,
  vanue.social.twitter.handle as twitter_handle,
  vanue.boxOfficeInfo.openHoursDetail as box_office_open_hours,
  vanue.boxOfficeInfo.acceptedPaymentDetail as box_office_accepted_payment,
  vanue.boxOfficeInfo.willCallDetail as box_office_will_call,
  vanue.parkingDetail as parking_detail,
  vanue.accessibleSeatingDetail as accessible_seating_detail,
  vanue.generalInfo.generalRule as general_rule,
  vanue.generalInfo.childRule as child_rule,
  vanue.ada.adaPhones as ada_phones,
  vanue.ada.adaCustomCopy as ada_custom_copy,
  vanue.ada.adaHours as ada_hours
from temp_venues
where vanue.test is false
and vanue.type = "venue"
"""
spark.sql(query)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:

query = f"""
alter table {env}_silver.event.venues 
alter column id set not null
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.venues 
add constraint pk_venues_id primary key (id)
"""
spark.sql(query)

DataFrame[]

In [0]:
query = f"""
create or replace table {env}_silver.event.event_venue as
  select distinct
          id_event, 
          vanue.id as id_venue
  from temp_venues
  where vanue.test is false
    and vanue.type = "venue"
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_venue 
add constraint fk_event_venue_event_id 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_venue 
add constraint fk_event_venue_venue_id 
foreign key (id_venue) 
references {env}_silver.event.venues(id)
"""
spark.sql(query)

DataFrame[]

## venue dma

In [0]:
query = f"""
create or replace table {env}_silver.event.event_dma
select distinct
  id_event, 
  dma.id as id_dma
from (
  select  id_event, 
          explode(vanue.dmas) as dma
  from temp_venues
  where vanue.test is false
  and vanue.type = "venue"
  )
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_dma 
add constraint fk_event_dma_event_id 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)


DataFrame[]

## venue market

In [0]:
query = f"""
create or replace table {env}_silver.event.market as 
select distinct market.id, market.name from (
select
  explode(vanue.markets) as market
from temp_venues
where vanue.test is false
and vanue.type = "venue"
and vanue.markets is not null
)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.market 
alter column id set not null
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.market 
add constraint pk_market_id primary key (id)
"""
spark.sql(query)


DataFrame[]

In [0]:
query = f"""
create or replace table {env}_silver.event.event_market as
    select distinct id_event, market.id as id_market from (
    select
    id_event,
    explode(vanue.markets) as market
    from temp_venues
    where vanue.test is false
    and vanue.type = "venue"
    and vanue.markets is not null)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_market 
add constraint fk_event_market_event_id 
foreign key (id_event) 
references {env}_silver.event.events(id)
"""
spark.sql(query)

query = f"""
alter table {env}_silver.event.event_market 
add constraint fk_event_market_market_id 
foreign key (id_market) 
references {env}_silver.event.market(id)
"""
spark.sql(query)

DataFrame[]